In [14]:
# spell correction
# p(correct|wrong) --> p(wrong|correct)*p(correct)

# language model -- bigram

from nltk.corpus import reuters

# read corpus
categories = reuters.categories()
corpus = reuters.sents(categories=categories)


# train bigram language model
term_count = {}
bigram_count = {}
for doc in corpus:
    doc = ['<s>'] + doc
    for i in range(0, len(doc)-1):
        # bigram: [i,i+1]
        term = doc[i]
        bigram = doc[i:i+2]
        
        if term in term_count:
            term_count[term]+=1
        else:
            term_count[term]=1
        bigram = ' '.join(bigram)
        if bigram in bigram_count:
            bigram_count[bigram]+=1
        else:
            bigram_count[bigram]=1

In [18]:
# 找出正确的词写成错误的词的概率  noisy channel model
channel_prob = {}

for line in open('spell-errors.txt'):
    items = line.split(":")
    correct = items[0].strip()
    mistakes = [item.strip() for item in items[1].strip().split(",")]
    channel_prob[correct] = {}
    for mis in mistakes:
        channel_prob[correct][mis] = 1.0/len(mistakes)

# print(channel_prob) 

In [26]:
# 错误的词可能是字典中哪些正确的词，用编辑距离计算
# 词典库
vocab = set([line.rstrip() for line in open('vocab.txt')])

def generate_candidates(word):
    # 只是用简便方法生成编辑距离为1的word
    key = 'abcdefghijklmnopqrstuvwxyz'
    
    candidate_list = []
    # insert
    for i in range(len(word)+1):
        if i >= len(word):
            for j in range(26):
                candidate_list.append(word[:i]+key[j])
        else:
            for j in range(26):
                candidate_list.append(word[:i]+key[j]+word[i:])
            
            
    # delete && replace
    for i in range(len(word)):
        if i+1 >= len(word):
            candidate_list.append(word[:i])
            for j in range(26):
                candidate_list.append(word[:i]+key[j])
        else:
            candidate_list.append(word[:i+1]+word[i+1:])
            for j in range(26):
                candidate_list.append(word[:i]+key[j]+word[i+1:])
                
    candidate = set(candidate_list)
    return [w for w in candidate if w in vocab]

generate_candidates("apple")

['apple', 'ample', 'apples', 'apply']

In [27]:
import numpy as np
V = len(term_count.keys())

file = open("testdata.txt", 'r')
for line in file:
    items = line.rstrip().split('\t')
    line = items[2].split()
    # line = ["I", "like", "playing"]
    for word in line:
        if word not in vocab:
            # 需要替换word成正确的单词
            # Step1: 生成所有的(valid)候选集合
            candidates = generate_candidates(word)
            
            # 一种方式： if candidate = [], 多生成几个candidates, 比如生成编辑距离不大于2的
            # TODO ： 根据条件生成更多的候选集合
            if len(candidates) < 1:
                continue   # 不建议这么做（这是不对的） 
            probs = []
            # 对于每一个candidate, 计算它的score
            # score = p(correct)*p(mistake|correct)
            #       = log p(correct) + log p(mistake|correct)
            # 返回score最大的candidate
            for candi in candidates:
                prob = 0
                # a. 计算channel probability
                if candi in channel_prob and word in channel_prob[candi]:
                    prob += np.log(channel_prob[candi][word])
                else:
                    prob += np.log(0.0001)
                
                # b. 计算语言模型的概率
                idx = items[2].index(word)+1
                if items[2][idx - 1] in bigram_count and candi in bigram_count[items[2][idx - 1]]:
                    prob += np.log((bigram_count[items[2][idx - 1]][candi] + 1.0) / (
                            term_count[bigram_count[items[2][idx - 1]]] + V))
                # TODO: 也要考虑当前 [word, post_word]
                #   prob += np.log(bigram概率)
                
                else:
                    prob += np.log(1.0 / V)

                probs.append(prob)
                
            max_idx = probs.index(max(probs))
            print (word, candidates[max_idx])

protectionst protectionist
products. products
long-run, long-run
gain. gain
17, 17e
retaiation retaliation
cost. costs
busines, business
ltMC.T. ltMC.T
U.S., U.S.
Murtha, Murtha
worried. worried
seriousnyss seriousness
aganst against
us, use
named. named
year, year
dlrs, dlrs
largest. largest
markets, markets
importsi imports
Products, Products
Retaliation, Retaliation
Group. Group
Korea's Koreans
Korea, Korea
Japan. Japan
U.S., U.S.
1985. 1985
Malaysia, Malaysia
Japn Japan
Kong, Kong
view. view
imports. imports
view, view
Lawrenc Lawrence
Mills, Mills
Industry. Industry
imports, imports
sources. sources
disadxantage disadvantage
trade, traded
said. said
market, markets
Friday. Friday
said. said
beef, beef
country. country
trad trade
continue. continues
Liberala Liberal
economy. economy
year. year
program. program
Representetive Representative
Kuroda, Kuroda
MITI, MITI
dispute. disputed
tonnes, tonnes
pct, pct
ootput output
rot, rot
tonnes, tonnes
pct, pct
vegetables. vegetables
waste,

shares. shares
spokesan spokesman
Ltd, Ltd
Ltd, Ltd
shares. shares
Ltd, Ltd
Ltd, Ltd
subsidiary. subsidiary
sale. sales
rates, rates
mawket market
months, months
Investors, Investors
said. said
marketsr markets
Australia, Australian
instruments, instruments
said. said
month, months
inflow. inflow
invesment investment
available, available
States. States
sloshinge sloshing
it. it
stability, stability
said. said
said. said
position, position
whemre wheare
money. money
Unit, Units
said. said
Dollar, Dollar
economost economist
said. said
encry entry
stable? stabled
policy, policy
said. said
economy, economy
said, said
ivestors investors
14. 14
elsewhere, elsewhere
Japanese, Japanese
said. said
moneye. moneyed
delivered, delivered
enough, enough
mght might
outflow, outflow
said. said
deficit, deficit
said. said
here, here
said. said
invetors investors
fluctuations. fluctuations
investors. investors
yen. yen
Dollar, Dollar
Ausfralian Australian
market. markets
followeg followed
markets, marke

shareholders, shareholders
proerly properly
financed, financed
said. said
provisions, provisions
Excoange Exchange
days, days
said. said
addition, addition
rquirement requirement
SEC. SEC
attempts. attempts
tnder tender
days. days
aquiring acquiring
offer. offers
takeovers, takeovers
enforced. enforced
stock, stock
Rather, Rather
raixe raise
money, money
said. said
requirements. requirements
takeover. takeovers
Pichens Pickens
it, it
said. said
treatmen. treatment
defenseve defensive
pills. pills
Proxnire Proxmire
spring. springs
Cengress Congress
year. year
22. 22
overdue, overdue
1987. 1987
1987, 1987
woul wool
dlrs. dlrs
payments. payments
15. 15
ECUS. ECUS
Stoltinberg Stoltenberg
levels. levels
today. today
agreement, agreements
accord, accord
underestimated. underestimated
ago, ago
Worlt Worst
Bank, Banks
partners. partners
accord, accord
said. said
it, it
here. here
agaenst against
dollar. dollar
spport support
dollar. dollar
said. said
growth. growth
said, said
however, however


businesses, businesses
lire, lire
prices, prices
said. said
Agrimont, Agrimont
Montedison, Montedison
exchange, exchange
said. said
mointained maintained
1986, 1986
said. said
Agrimont. Agrimont
Picco, Picco
Ceroni, Ceroni
Sifi, Sifi
board, boards
said. said
transaction, transactions
Datro Datron
stock, stock
officers, officers
share, shared
said. said
totali total
outstanding. outstanding
transaction, transactions
officors officers
company. company
plan, plant
Detron Datron
said. said
July, July
complected compleated
31, 31
said. said
appicable applicable
taxes, taxes
said. said
Interstate, Interstate
adout about
stock, stock
acquisition. acquisitions
rech reach
offered, offered
said. said
tenred tended
Tunisia, Tunisian
said. said
wteat wheat
credits. credits
amoun, amount
1, 1a
said. said
refect reject
plan. plant
Eritions Editions
said, said
however, however
contyol control
years. years
parent, parent
ltHBJ, ltHBJ
October. October
Burlington, Burlington
stake. stakes
publishedn pub

plans. plans
Today, Today
circulaed circulated
stock. stock
juped duped
Unites unites
cnsider insider
tink think
proposel propose
it, it
Pettee, Pettee
Steans Stearns
analyst. analyst
Anlysts Analysts
airline. airlines
adted acted
cars, cars
airline. airlines
boginning beginning
end. end
emphasizin emphasizing
one. ones
valies varies
share. shared
todayl today
yubing tubing
widespread, widespread
belive believe
play. play
catalyst, catalysts
anoteer another
buyer. buyer
airlinee airlines
past, pasty
Pettee. Pettee
intersting interesting
valies varies
ther. there
everybody, everybody
said. said
reterated reiterated
earningc earning
goal. goal
1986, 1986
30, 30
1986, 1986
Harmarx Hartmarx
dlrs, dlrs
share, shared
dlrs, dlrs
share. shared
manuacturer manufacturer
share, shared
ago. ago
1987, 1987
Detroit, Detroit
Louis, Louis
Francisco. Francisco
continuoes continuous
grow, growl
ties, ties
clothing. clothing
agency, agency
London, London
Asamoah. Asamoah
exploitation, exploitation
refini

In [11]:
# 解决缺失punkt文件的问题

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')


[nltk_data] Downloading package punkt to E:\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True